<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

En este notebook aprenderás qué es una **variable categórica**, junto con tres enfoques para manejar este tipo de datos.

# Introducción

Una **variable categórica** solo puede tomar un número limitado de valores.

- Por ejemplo, imagina una encuesta que pregunta con qué frecuencia desayunas, con las siguientes opciones: "Nunca", "Raramente", "Casi todos los días" o "Todos los días". En este caso, los datos son categóricos porque las respuestas caen dentro de un conjunto fijo de categorías.
- Si se preguntara en una encuesta qué marca de automóvil posee una persona, las respuestas caerían en categorías como "Honda", "Toyota" o "Ford". En este caso, los datos también serían categóricos.

Obtendrás un error si intentas introducir estas variables en la mayoría de modelos de machine learning en Python sin preprocesarlas antes. En este tutorial, compararemos tres enfoques para preparar tus datos categóricos.

# Tres Enfoques

### 1) Eliminar las variables categóricas

El enfoque más sencillo para manejar variables categóricas es simplemente eliminarlas del conjunto de datos.  
Este enfoque solo funcionará bien si las columnas eliminadas no contenían información útil.

### 2) Codificación ordinal

La **codificación ordinal** asigna a cada valor único un número entero diferente.

![tut3_ordinalencode](https://storage.googleapis.com/kaggle-media/learn/images/tEogUAr.png)

Este enfoque asume un orden en las categorías: "Nunca" (0) < "Raramente" (1) < "Casi todos los días" (2) < "Todos los días" (3).

Esta suposición tiene sentido en este ejemplo, ya que hay una jerarquía clara en las categorías.  
No todas las variables categóricas tienen un orden natural en sus valores, pero aquellas que sí lo tienen se denominan **variables ordinales**.  
En modelos basados en árboles (como árboles de decisión o bosques aleatorios), se espera que la codificación ordinal funcione bien con variables ordinales.

### 3) Codificación one-hot

La **codificación one-hot** (o "codificación de una sola posición activa") crea nuevas columnas que indican la presencia (o ausencia) de cada valor posible en los datos originales. Veamos un ejemplo para entenderlo mejor:

![tut3_onehot](https://storage.googleapis.com/kaggle-media/learn/images/TW5m0aJ.png)

En el conjunto de datos original, "Color" es una variable categórica con tres categorías: "Rojo", "Amarillo" y "Verde".  
La codificación one-hot correspondiente contiene una columna para cada valor posible y una fila por cada observación en el conjunto de datos original.  
Cuando el valor original era "Rojo", colocamos un 1 en la columna "Rojo"; si era "Amarillo", un 1 en la columna "Amarillo", y así sucesivamente.

A diferencia de la codificación ordinal, la codificación one-hot **no asume** un orden entre las categorías. Por ello, este enfoque funciona especialmente bien cuando **no hay un orden natural** en los datos categóricos (por ejemplo, "Rojo" no es ni _más_ ni _menos_ que "Amarillo").  
A este tipo de variables categóricas sin jerarquía intrínseca se les conoce como **variables nominales**.

Sin embargo, la codificación one-hot no funciona bien cuando la variable categórica toma un número elevado de valores diferentes (generalmente, no se recomienda para más de 15 categorías únicas).

# Ejemplo

Como en el tutorial anterior, trabajaremos con el [conjunto de datos de viviendas de Melbourne](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home).  

No nos centraremos en la etapa de carga de datos. En su lugar, supondremos que ya cuentas con los datos de entrenamiento y validación en las variables `X_train`, `X_valid`, `y_train` y `y_valid`.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer los datos
data = pd.read_csv('/content/melb_data.csv')

# Separar la variable objetivo de los predictores
y = data.Price
X = data.drop(['Price'], axis=1)

# Dividir los datos en subconjuntos de entrenamiento y validación
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Eliminar columnas con valores faltantes (el enfoque más simple)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinalidad" se refiere al número de valores únicos en una columna
# Seleccionar columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitrario)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

# Seleccionar columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Conservar solo las columnas seleccionadas
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento utilizando el método `head()` a continuación.

In [ ]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


A continuación, obtenemos una lista de todas las variables categóricas presentes en los datos de entrenamiento.

Para ello, verificamos el tipo de dato (o **dtype**) de cada columna. El tipo de dato `object` indica que una columna contiene texto (teóricamente podría ser otro tipo de dato, pero eso no es relevante en nuestro caso).  
En este conjunto de datos, las columnas con texto corresponden a variables categóricas.

In [4]:
# Obtener la lista de variables categóricas
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Variables categóricas:")
print(object_cols)

Variables categóricas:
['Type', 'Method', 'Regionname']


### Definimos una función para medir la calidad de cada enfoque

Definimos la función `score_dataset()` para comparar los tres enfoques distintos de tratamiento de variables categóricas. Esta función reporta el [error absoluto medio (MAE)](https://es.wikipedia.org/wiki/Error_absoluto_medio) obtenido a partir de un modelo de bosque aleatorio (*random forest*).  
En general, ¡queremos que el MAE sea lo más bajo posible!


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Puntuación del Enfoque 1 (Eliminar variables categóricas)

Eliminamos las columnas de tipo `object` utilizando el método [`select_dtypes()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html).


In [6]:
# Eliminar las columnas categóricas (tipo 'object')
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE del Enfoque 1 (Eliminar variables categóricas):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))


MAE del Enfoque 1 (Eliminar variables categóricas):
175703.48185157913


### Puntuación del Enfoque 2 (Codificación ordinal)

Scikit-learn proporciona la clase [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html), que puede utilizarse para obtener codificaciones ordinales.  
Recorremos las variables categóricas y aplicamos el codificador ordinal de forma separada a cada columna.



In [7]:
from sklearn.preprocessing import OrdinalEncoder

# Crear una copia para evitar modificar los datos originales
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Aplicar codificación ordinal a cada columna con datos categóricos
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE del Enfoque 2 (Codificación ordinal):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE del Enfoque 2 (Codificación ordinal):
165936.40548390493


En la celda de código anterior, para cada columna, asignamos aleatoriamente un valor entero distinto a cada valor único. Este es un enfoque común y más simple que proporcionar etiquetas personalizadas; sin embargo, podríamos obtener un mejor rendimiento si asignamos etiquetas más informadas para todas las variables ordinales.

### Puntuación del Enfoque 3 (Codificación One-Hot)

Utilizamos la clase [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) de scikit-learn para obtener las codificaciones one-hot. Esta clase permite varios parámetros para personalizar su comportamiento:  
- Establecemos `handle_unknown='ignore'` para evitar errores cuando los datos de validación contienen categorías que no están presentes en los datos de entrenamiento, y  
- establecemos `sparse_output=False` para asegurarnos de que las columnas codificadas se devuelvan como un arreglo de NumPy (en lugar de una matriz dispersa).

Para usar el codificador, proporcionamos únicamente las columnas categóricas que queremos codificar.  
Por ejemplo, para codificar los datos de entrenamiento, utilizamos `X_train[object_cols]`. (`object_cols` en la celda de código siguiente es una lista con los nombres de las columnas que contienen datos categóricos; por tanto, `X_train[object_cols]` contiene todos los datos categóricos del conjunto de entrenamiento.)


In [11]:
from sklearn.preprocessing import OneHotEncoder

# Aplicar codificación one-hot a cada columna con datos categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# La codificación one-hot eliminó los índices; los restauramos
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Eliminamos las columnas categóricas (que serán reemplazadas por su codificación one-hot)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Añadimos las columnas codificadas one-hot a las características numéricas
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Aseguremonos de que todos los nombres de columnas sean tipo string
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE del Enfoque 3 (Codificación One-Hot):")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))


MAE del Enfoque 3 (Codificación One-Hot):
166089.4893009678


# ¿Qué enfoque es el mejor?

En este caso, eliminar las columnas categóricas (**Enfoque 1**) fue el que tuvo peor desempeño, ya que presentó el mayor valor de MAE. En cuanto a los otros dos enfoques, dado que los valores de MAE son muy similares, no parece haber una ventaja significativa de uno sobre el otro.

En general, la codificación one-hot (**Enfoque 3**) suele ofrecer el mejor rendimiento, mientras que eliminar las variables categóricas (**Enfoque 1**) tiende a ser el menos efectivo. Sin embargo, esto puede variar dependiendo del conjunto de datos específico.

# Conclusión

El mundo está lleno de datos categóricos. ¡Serás un científico de datos mucho más eficaz si sabes cómo tratar con este tipo de datos tan común!